In [1]:
!pip install scikit-multilearn

     |████████████████████████████████| 92kB 2.4MB/s 


In [2]:
import pandas as pd
import matplotlib as plt
import os
import csv
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.preprocessing import StandardScaler

from skmultilearn.problem_transform import LabelPowerset
from imblearn.over_sampling import RandomOverSampler
from imblearn.under_sampling import RandomUnderSampler
from sklearn.model_selection import train_test_split, cross_val_score,StratifiedKFold
from skmultilearn.problem_transform import BinaryRelevance, ClassifierChain, LabelPowerset
from sklearn.metrics import accuracy_score,f1_score,confusion_matrix,classification_report

from sklearn.linear_model import SGDClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import NuSVC, LinearSVC, SVC

import pickle
from scipy.sparse import hstack
from skmultilearn.model_selection import iterative_train_test_split

/usr/local/lib/python3.6/dist-packages/sklearn/externals/six.py:31: FutureWarning: The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).
  "(https://pypi.org/project/six/).", FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.neighbors.base module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.neighbors. Anything that cannot be imported from sklearn.neighbors is now part of the private API.
  warnings.warn(message, FutureWarning)


In [3]:
# Mount Drive
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [4]:
dir = "drive/My Drive/BT4221/Data/"

# cleaned train dataset
df = pd.read_csv(dir+"cleaned_train.csv")
# for EDA
data_raw = pd.read_csv(dir+"train.csv")

df.head()

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate,word_count,char_count,word_density,total_length,capitals,capitals_prop,num_exclamation_marks,num_question_marks,num_punctuation,num_symbols,num_unique_words,prop_unique_words,processed_text,polarity,subjectivity
0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0,43,223,0.191964,264,17,0.064394,0,1,6,0,41,0.953488,explanation edits made username hardcore metal...,0.136364,0.454545
1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0,17,95,0.177083,112,8,0.071429,1,0,5,0,17,1.000000,daww match background colour seemingly stuck t...,0.200000,0.200000
2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0,42,192,0.217617,233,4,0.017167,0,0,4,0,39,0.928571,hey man really trying edit war guy constantly ...,0.150000,0.383333
3,0001b41b1c6bb37e,"""\nMore\nI can't make any real suggestions on ...",0,0,0,0,0,0,113,511,0.220703,622,11,0.017685,0,0,6,0,82,0.725664,ca nt make real suggestion improvement wondere...,0.183333,0.297222
4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0,13,55,0.232143,67,2,0.029851,0,1,3,0,13,1.000000,sir hero chance remember page,0.000000,0.000000


In [5]:
count_vect = pickle.load(open('/content/drive/My Drive/BT4221/Models/count_vect_v1.sav','rb'))
tf = pickle.load(open('/content/drive/My Drive/BT4221/Models/tf_v1.sav','rb'))

In [6]:
X= df['processed_text']
count_vect = CountVectorizer()
X_count = count_vect.fit_transform(X)
tf = TfidfTransformer()
X = tf.fit_transform(X_count)

# Preprocessing

In [ ]:
added_features = pd.DataFrame(df.iloc[:, 8:23])
added_features.drop(columns='processed_text',inplace=True)
for column in added_features.columns:
    var_sparse = added_features[column].values[:, None]
    X = hstack((X, var_sparse.astype(float)),format='csr')
print(X.shape)

(159502, 188840)


In [ ]:
y = pd.DataFrame(df.iloc[:, 2:8]) #obtain labels dataframe
#X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)
from sklearn.model_selection import ShuffleSplit

skf = ShuffleSplit(n_splits=1,test_size=0.2, random_state=42)
for train_id, test_id in skf.split(X,y):
    X_train, X_val = X[train_id], X[test_id]
    y_train, y_val = y.loc[train_id,:], y.loc[test_id,:]
    X_val_original = df['comment_text'][test_id]
X_val_original

63002                 fuck you bizitch \n\nYo fuck u fuck u
5813      "HUNK should have his own page separate from m...
26667     "\n Why do you classify the Liberal Democrats ...
148285    , 15 September 2013 (UTC)\nI have already agre...
57628     "\nGave a couple stories a listen and the repo...
                                ...                        
80579     "\n\nThat link you are looking for is now in t...
65660     "\n\nPersonally, I am shocked that this sort o...
138620    Any notional improvement was purely accidental...
36796     Good evening, Terrence! \n\nIn the UK, the tim...
139959    Oh not you again!!! And so? After the block ha...
Name: comment_text, Length: 31901, dtype: object

# SGD

In [ ]:
#!pip install scikit-multilearn

model = SGDClassifier(alpha=1e-05, average=False, class_weight=None,
              early_stopping=False, epsilon=0.1, eta0=0.0, fit_intercept=True,
              l1_ratio=0.15, learning_rate='optimal', loss='modified_huber',
              max_iter=50, n_iter_no_change=5, n_jobs=None, penalty='l2',
              power_t=0.5, random_state=None, shuffle=True, tol=0.001,
              validation_fraction=0.1, verbose=0, warm_start=False)

clf_BR = BinaryRelevance(#treat each label as a separate single-class classification problems
    classifier = model,
    require_dense = [False, True]
)

clf_CC = ClassifierChain(#treat each label as a park of a conditioned chain of single-class classificatn prob
    classifier = model,
    require_dense = [False, True]
)


clf_LP = LabelPowerset(
    classifier = model,
    require_dense = [False, True]
)

In [ ]:
clf_BR.fit(X_train, y_train)

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_stochastic_gradient.py:557: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  ConvergenceWarning)


BinaryRelevance(classifier=SGDClassifier(alpha=1e-05, average=False,
                                         class_weight=None,
                                         early_stopping=False, epsilon=0.1,
                                         eta0=0.0, fit_intercept=True,
                                         l1_ratio=0.15, learning_rate='optimal',
                                         loss='modified_huber', max_iter=50,
                                         n_iter_no_change=5, n_jobs=None,
                                         penalty='l2', power_t=0.5,
                                         random_state=None, shuffle=True,
                                         tol=0.001, validation_fraction=0.1,
                                         verbose=0, warm_start=False),
                require_dense=[False, True])

In [ ]:
clf_CC.fit(X_train,y_train)

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_stochastic_gradient.py:557: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  ConvergenceWarning)


ClassifierChain(classifier=SGDClassifier(alpha=1e-05, average=False,
                                         class_weight=None,
                                         early_stopping=False, epsilon=0.1,
                                         eta0=0.0, fit_intercept=True,
                                         l1_ratio=0.15, learning_rate='optimal',
                                         loss='modified_huber', max_iter=50,
                                         n_iter_no_change=5, n_jobs=None,
                                         penalty='l2', power_t=0.5,
                                         random_state=None, shuffle=True,
                                         tol=0.001, validation_fraction=0.1,
                                         verbose=0, warm_start=False),
                order=None, require_dense=[False, True])

In [ ]:
clf_LP.fit(X_train,y_train)

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_stochastic_gradient.py:557: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  ConvergenceWarning)


LabelPowerset(classifier=SGDClassifier(alpha=1e-05, average=False,
                                       class_weight=None, early_stopping=False,
                                       epsilon=0.1, eta0=0.0,
                                       fit_intercept=True, l1_ratio=0.15,
                                       learning_rate='optimal',
                                       loss='modified_huber', max_iter=50,
                                       n_iter_no_change=5, n_jobs=None,
                                       penalty='l2', power_t=0.5,
                                       random_state=None, shuffle=True,
                                       tol=0.001, validation_fraction=0.1,
                                       verbose=0, warm_start=False),
              require_dense=[False, True])

In [ ]:
pickle.dump(clf_BR,open('/content/drive/My Drive/BT4221/Models/BR_SGD.sav','wb'))
pickle.dump(clf_CC,open('/content/drive/My Drive/BT4221/Models/CC_SGD.sav','wb'))
pickle.dump(clf_LP,open('/content/drive/My Drive/BT4221/Models/LP_SGD.sav','wb'))

In [ ]:
prediction_BR = clf_BR.predict(X_val)
prediction_CC = clf_CC.predict(X_val)
prediction_LP = clf_LP.predict(X_val)

In [ ]:
 #The fraction of labels that are incorrectly predicted
import sklearn.metrics as metrics

print("Hamming Loss (Binary Relevance) : {}".format(metrics.hamming_loss(y_val, prediction_BR)))
print("Hamming Loss (Classification Chain) : {}".format(metrics.hamming_loss(y_val, prediction_CC)))
print("Hamming Loss (Label Powerset) : {}".format(metrics.hamming_loss(y_val, prediction_LP)))

Hamming Loss (Binary Relevance) : 0.18771093905102243
Hamming Loss (Classification Chain) : 0.04148772765744021
Hamming Loss (Label Powerset) : 0.037433518280513674


In [ ]:
print("Accuracy Score (Binary Relevance) : {}".format(metrics.accuracy_score(y_val, prediction_BR)))
print("Accuracy Score (Classification Chain) : {}".format(metrics.accuracy_score(y_val, prediction_CC)))
print("Accuracy Score (Label Powerset) : {}".format(metrics.accuracy_score(y_val, prediction_LP)))

Accuracy Score (Binary Relevance) : 0.013103037522334724
Accuracy Score (Classification Chain) : 0.8847371555750604
Accuracy Score (Label Powerset) : 0.897275947462462


In [ ]:
print("F1 Score (Binary Relevance) : {}".format(f1_score(prediction_BR,y_val,average='micro')))
print("F1 Score (Classification Chain) : {}".format(f1_score(prediction_CC,y_val,average='micro')))
print("F1 Score (Label Powerset) : {}".format(f1_score(prediction_LP,y_val,average='micro')))

F1 Score (Binary Relevance) : 0.1413788983152109
F1 Score (Classification Chain) : 0.2854314766489697
F1 Score (Label Powerset) : 0.0


# Iterative Stratification

In [ ]:
# y = pd.DataFrame(df.iloc[:, 2:8]) 
# indices = np.arange(X.shape[0])
y = df[["toxic", "severe_toxic", "obscene", "threat", "insult", "identity_hate"]].values
X_train, y_train, X_test, y_test = iterative_train_test_split(X, y, test_size = 0.2)

In [ ]:
model = SGDClassifier(alpha=1e-05, average=False, class_weight=None,
              early_stopping=False, epsilon=0.1, eta0=0.0, fit_intercept=True,
              l1_ratio=0.15, learning_rate='optimal', loss='modified_huber',
              max_iter=100, n_iter_no_change=5, n_jobs=None, penalty='l2',
              power_t=0.5, random_state=None, shuffle=True, tol=0.001,
              validation_fraction=0.1, verbose=0, warm_start=False)

clf_BR = BinaryRelevance(#treat each label as a separate single-class classification problems
    classifier = model,
    require_dense = [False, True]
)

clf_CC = ClassifierChain(#treat each label as a park of a conditioned chain of single-class classificatn prob
    classifier = model,
    require_dense = [False, True]
)


clf_LP = LabelPowerset(
    classifier = model,
    require_dense = [False, True]
)

In [ ]:
clf_BR.fit(X_train,y_train)
clf_CC.fit(X_train,y_train)
clf_LP.fit(X_train,y_train)

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_stochastic_gradient.py:557: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  ConvergenceWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_stochastic_gradient.py:557: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  ConvergenceWarning)


LabelPowerset(classifier=SGDClassifier(alpha=1e-05, average=False,
                                       class_weight=None, early_stopping=False,
                                       epsilon=0.1, eta0=0.0,
                                       fit_intercept=True, l1_ratio=0.15,
                                       learning_rate='optimal',
                                       loss='modified_huber', max_iter=100,
                                       n_iter_no_change=5, n_jobs=None,
                                       penalty='l2', power_t=0.5,
                                       random_state=None, shuffle=True,
                                       tol=0.001, validation_fraction=0.1,
                                       verbose=0, warm_start=False),
              require_dense=[False, True])

In [ ]:
prediction_BR = clf_BR.predict(X_test)
prediction_CC = clf_CC.predict(X_test)
prediction_LP = clf_LP.predict(X_test)

In [ ]:
 #The fraction of labels that are incorrectly predicted
import sklearn.metrics as metrics

print("Hamming Loss (Binary Relevance) : {}".format(metrics.hamming_loss(y_test, prediction_BR)))
print("Hamming Loss (Classification Chain) : {}".format(metrics.hamming_loss(y_test, prediction_CC)))
print("Hamming Loss (Label Powerset) : {}".format(metrics.hamming_loss(y_test, prediction_LP)))

Hamming Loss (Binary Relevance) : 0.03718274244276564
Hamming Loss (Classification Chain) : 0.03854633606052057
Hamming Loss (Label Powerset) : 0.04918863567495272


In [ ]:
print("Accuracy Score (Binary Relevance) : {}".format(metrics.accuracy_score(y_test, prediction_BR)))
print("Accuracy Score (Classification Chain) : {}".format(metrics.accuracy_score(y_test, prediction_CC)))
print("Accuracy Score (Label Powerset) : {}".format(metrics.accuracy_score(y_test, prediction_LP)))

Accuracy Score (Binary Relevance) : 0.8761794301119087
Accuracy Score (Classification Chain) : 0.8882793642832513
Accuracy Score (Label Powerset) : 0.8527005423027492


In [ ]:
print("F1 Score (Binary Relevance) : {}".format(f1_score(prediction_BR,y_test,average='micro')))
print("F1 Score (Classification Chain) : {}".format(f1_score(prediction_CC,y_test,average='micro')))
print("F1 Score (Label Powerset) : {}".format(f1_score(prediction_LP,y_test,average='micro')))

F1 Score (Binary Relevance) : 0.2872308462694041
F1 Score (Classification Chain) : 0.22205820328975118
F1 Score (Label Powerset) : 0.23523678011534402


In [ ]:
pickle.dump(clf_BR,open('/content/drive/My Drive/BT4221/Models/BR_SVM.sav','wb'))
pickle.dump(clf_CC,open('/content/drive/My Drive/BT4221/Models/CC_SVM.sav','wb'))
pickle.dump(clf_LP,open('/content/drive/My Drive/BT4221/Models/LP_SVM.sav','wb'))

# Logistic Regression

In [ ]:
model = LogisticRegression(multi_class='auto', penalty='l2', max_iter=500, class_weight='balanced', dual=False)

clf_BR = BinaryRelevance(#treat each label as a separate single-class classification problems
    classifier = model,
    require_dense = [False, True]
)

clf_CC = ClassifierChain(#treat each label as a park of a conditioned chain of single-class classificatn prob
    classifier = model,
    require_dense = [False, True]
)


clf_LP = LabelPowerset(
    classifier = model,
    require_dense = [False, True]
)

In [ ]:
clf_BR.fit(X_train, y_train)
clf_CC.fit(X_train,y_train)
clf_LP.fit(X_train,y_train)

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression


LabelPowerset(classifier=LogisticRegression(C=1.0, class_weight='balanced',
                                            dual=False, fit_intercept=True,
                                            intercept_scaling=1, l1_ratio=None,
                                            max_iter=500, multi_class='auto',
                                            n_jobs=None, penalty='l2',
                                            random_state=None, solver='lbfgs',
                                            tol=0.0001, verbose=0,
                                            warm_start=False),
              require_dense=[False, True])

In [ ]:
prediction_BR = clf_BR.predict(X_val)
prediction_CC = clf_CC.predict(X_val)
prediction_LP = clf_LP.predict(X_val)

In [ ]:
 #The fraction of labels that are incorrectly predicted
import sklearn.metrics as metrics

print("Hamming Loss (Binary Relevance) : {}".format(metrics.hamming_loss(y_val, prediction_BR)))
print("Hamming Loss (Classification Chain) : {}".format(metrics.hamming_loss(y_val, prediction_CC)))
print("Hamming Loss (Label Powerset) : {}".format(metrics.hamming_loss(y_val, prediction_LP)))

Hamming Loss (Binary Relevance) : 0.12170464875709225
Hamming Loss (Classification Chain) : 0.18296187162366906
Hamming Loss (Label Powerset) : 0.40976249438366613


In [ ]:
print("Accuracy Score (Binary Relevance) : {}".format(metrics.accuracy_score(y_val, prediction_BR)))
print("Accuracy Score (Classification Chain) : {}".format(metrics.accuracy_score(y_val, prediction_CC)))
print("Accuracy Score (Label Powerset) : {}".format(metrics.accuracy_score(y_val, prediction_LP)))

Accuracy Score (Binary Relevance) : 0.6950565812983919
Accuracy Score (Classification Chain) : 0.764834958151782
Accuracy Score (Label Powerset) : 0.008338296605122097


In [ ]:
pickle.dump(clf_BR,open('/content/drive/My Drive/BT4221/Models/BR_LR.sav','wb'))
pickle.dump(clf_CC,open('/content/drive/My Drive/BT4221/Models/CC_LR.sav','wb'))
pickle.dump(clf_LP,open('/content/drive/My Drive/BT4221/Models/LP_LR.sav','wb'))

# Random Forest

In [ ]:
from sklearn.ensemble import RandomForestClassifier

model = RandomForestClassifier(min_samples_split = 2, class_weight='balanced')

clf_BR = BinaryRelevance(#treat each label as a separate single-class classification problems
    classifier = model,
    require_dense = [False, True]
)

clf_CC = ClassifierChain(#treat each label as a park of a conditioned chain of single-class classificatn prob
    classifier = model,
    require_dense = [False, True]
)


clf_LP = LabelPowerset(
    classifier = model,
    require_dense = [False, True]
)

NameError: ignored

In [ ]:
clf_BR.fit(X_train, y_train)
clf_CC.fit(X_train,y_train)
clf_LP.fit(X_train,y_train)

In [ ]:
prediction_BR = clf_BR.predict(X_val)
prediction_CC = clf_CC.predict(X_val)
prediction_LP = clf_LP.predict(X_val)

In [ ]:
 #The fraction of labels that are incorrectly predicted
import sklearn.metrics as metrics

print("Hamming Loss (Binary Relevance) : {}".format(metrics.hamming_loss(y_val, prediction_BR)))
print("Hamming Loss (Classification Chain) : {}".format(metrics.hamming_loss(y_val, prediction_CC)))
print("Hamming Loss (Label Powerset) : {}".format(metrics.hamming_loss(y_val, prediction_LP)))

In [ ]:
print("Accuracy Score (Binary Relevance) : {}".format(metrics.accuracy_score(y_val, prediction_BR)))
print("Accuracy Score (Classification Chain) : {}".format(metrics.accuracy_score(y_val, prediction_CC)))
print("Accuracy Score (Label Powerset) : {}".format(metrics.accuracy_score(y_val, prediction_LP)))

In [ ]:
pickle.dump(clf_BR,open('/content/drive/My Drive/BT4221/Models/BR_RF.sav','wb'))
pickle.dump(clf_CC,open('/content/drive/My Drive/BT4221/Models/CC_RF.sav','wb'))
pickle.dump(clf_LP,open('/content/drive/My Drive/BT4221/Models/LP_RF.sav','wb'))

# Normalization

In [7]:
added_features = pd.DataFrame(df.iloc[:, 8:23])
added_features.drop(columns='processed_text',inplace=True)
for column in added_features.columns:
    var_sparse = StandardScaler().fit_transform(added_features[column].values[:, None])
    X = hstack((X, var_sparse.astype(float)),format='csr')
print(X.shape)

(159502, 188840)


In [8]:
y = df[["toxic", "severe_toxic", "obscene", "threat", "insult", "identity_hate"]].values
X_train, y_train, X_test, y_test = iterative_train_test_split(X, y, test_size = 0.2)

## SGD

In [ ]:
model = SGDClassifier(alpha=1e-05, average=False, class_weight=None,
              early_stopping=False, epsilon=0.1, eta0=0.0, fit_intercept=True,
              l1_ratio=0.15, learning_rate='optimal', loss='modified_huber',
              max_iter=50, n_iter_no_change=5, n_jobs=None, penalty='l2',
              power_t=0.5, random_state=None, shuffle=True, tol=0.001,
              validation_fraction=0.1, verbose=0, warm_start=False)

clf_BR = BinaryRelevance(#treat each label as a separate single-class classification problems
    classifier = model,
    require_dense = [False, True]
)

clf_CC = ClassifierChain(#treat each label as a park of a conditioned chain of single-class classificatn prob
    classifier = model,
    require_dense = [False, True]
)


clf_LP = LabelPowerset(
    classifier = model,
    require_dense = [False, True]
)

In [ ]:
clf_BR.fit(X_train,y_train)
clf_CC.fit(X_train,y_train)
clf_LP.fit(X_train,y_train)

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_stochastic_gradient.py:557: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  ConvergenceWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_stochastic_gradient.py:557: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  ConvergenceWarning)


LabelPowerset(classifier=SGDClassifier(alpha=1e-05, average=False,
                                       class_weight=None, early_stopping=False,
                                       epsilon=0.1, eta0=0.0,
                                       fit_intercept=True, l1_ratio=0.15,
                                       learning_rate='optimal',
                                       loss='modified_huber', max_iter=50,
                                       n_iter_no_change=5, n_jobs=None,
                                       penalty='l2', power_t=0.5,
                                       random_state=None, shuffle=True,
                                       tol=0.001, validation_fraction=0.1,
                                       verbose=0, warm_start=False),
              require_dense=[False, True])

In [ ]:
prediction_BR = clf_BR.predict(X_test)
prediction_CC = clf_CC.predict(X_test)
prediction_LP = clf_LP.predict(X_test)

In [ ]:
import sklearn.metrics as metrics

print("Hamming Loss (Binary Relevance) : {}".format(metrics.hamming_loss(y_test, prediction_BR)))
print("Hamming Loss (Classification Chain) : {}".format(metrics.hamming_loss(y_test, prediction_CC)))
print("Hamming Loss (Label Powerset) : {}".format(metrics.hamming_loss(y_test, prediction_LP)))

Hamming Loss (Binary Relevance) : 0.022789254255352498
Hamming Loss (Classification Chain) : 0.024753664984378754
Hamming Loss (Label Powerset) : 0.031195469316531353


In [ ]:
print("Accuracy Score (Binary Relevance) : {}".format(metrics.accuracy_score(y_test, prediction_BR)))
print("Accuracy Score (Classification Chain) : {}".format(metrics.accuracy_score(y_test, prediction_CC)))
print("Accuracy Score (Label Powerset) : {}".format(metrics.accuracy_score(y_test, prediction_LP)))

Accuracy Score (Binary Relevance) : 0.905049998432651
Accuracy Score (Classification Chain) : 0.9067740823171687
Accuracy Score (Label Powerset) : 0.8961788031723144


In [ ]:
print("F1 Score (Binary Relevance) : {}".format(f1_score(prediction_BR,y_test,average='micro')))
print("F1 Score (Classification Chain) : {}".format(f1_score(prediction_CC,y_test,average='micro')))
print("F1 Score (Label Powerset) : {}".format(f1_score(prediction_LP,y_test,average='micro')))

F1 Score (Binary Relevance) : 0.6518199233716476
F1 Score (Classification Chain) : 0.6619095190523762
F1 Score (Label Powerset) : 0.3815639564992232


In [ ]:
pickle.dump(clf_BR,open('/content/drive/My Drive/BT4221/Models/BR_SGD_v3.sav','wb'))
pickle.dump(clf_CC,open('/content/drive/My Drive/BT4221/Models/CC_SGD_v3.sav','wb'))
pickle.dump(clf_LP,open('/content/drive/My Drive/BT4221/Models/LP_SGD_v3.sav','wb'))

## Logistic Regression

In [ ]:
model = LogisticRegression(multi_class='auto', penalty='l2', max_iter=200, class_weight='balanced', dual=False)

clf_BR = BinaryRelevance(#treat each label as a separate single-class classification problems
    classifier = model,
    require_dense = [False, True]
)

clf_CC = ClassifierChain(#treat each label as a park of a conditioned chain of single-class classificatn prob
    classifier = model,
    require_dense = [False, True]
)


clf_LP = LabelPowerset(
    classifier = model,
    require_dense = [False, True]
)

In [ ]:
clf_BR.fit(X_train,y_train)
clf_CC.fit(X_train,y_train)
clf_LP.fit(X_train,y_train)

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression


LabelPowerset(classifier=LogisticRegression(C=1.0, class_weight='balanced',
                                            dual=False, fit_intercept=True,
                                            intercept_scaling=1, l1_ratio=None,
                                            max_iter=200, multi_class='auto',
                                            n_jobs=None, penalty='l2',
                                            random_state=None, solver='lbfgs',
                                            tol=0.0001, verbose=0,
                                            warm_start=False),
              require_dense=[False, True])

In [ ]:
prediction_BR = clf_BR.predict(X_test)
prediction_CC = clf_CC.predict(X_test)
prediction_LP = clf_LP.predict(X_test)

In [ ]:
#import sklearn.metrics as metrics

print("Hamming Loss (Binary Relevance) : {}".format(metrics.hamming_loss(y_test, prediction_BR)))
print("Hamming Loss (Classification Chain) : {}".format(metrics.hamming_loss(y_test, prediction_CC)))
print("Hamming Loss (Label Powerset) : {}".format(metrics.hamming_loss(y_test, prediction_LP)))

print("Accuracy Score (Binary Relevance) : {}".format(metrics.accuracy_score(y_test, prediction_BR)))
print("Accuracy Score (Classification Chain) : {}".format(metrics.accuracy_score(y_test, prediction_CC)))
print("Accuracy Score (Label Powerset) : {}".format(metrics.accuracy_score(y_test, prediction_LP)))

print("F1 Score (Binary Relevance) : {}".format(f1_score(prediction_BR,y_test,average='micro')))
print("F1 Score (Classification Chain) : {}".format(f1_score(prediction_CC,y_test,average='micro')))
print("F1 Score (Label Powerset) : {}".format(f1_score(prediction_LP,y_test,average='micro')))

Hamming Loss (Binary Relevance) : 0.03050061126610451
Hamming Loss (Classification Chain) : 0.05401607055160235
Hamming Loss (Label Powerset) : 0.07098523557255258
Accuracy Score (Binary Relevance) : 0.87824833077333
Accuracy Score (Classification Chain) : 0.8660543556628318
Accuracy Score (Label Powerset) : 0.6985360960471458
F1 Score (Binary Relevance) : 0.6767799800686525
F1 Score (Classification Chain) : 0.5527920757818245
F1 Score (Label Powerset) : 0.42160827550976976


In [ ]:
pickle.dump(clf_BR,open('/content/drive/My Drive/BT4221/Models/BR_LR_v3.sav','wb'))
pickle.dump(clf_CC,open('/content/drive/My Drive/BT4221/Models/CC_LR_v3.sav','wb'))
pickle.dump(clf_LP,open('/content/drive/My Drive/BT4221/Models/LP_LR_v3.sav','wb'))

## SVM

In [9]:
model = SVC(probability=True, C=0.5, kernel='rbf', 
            decision_function_shape= 'ovr',degree=3, gamma=0.5,shrinking= True)

clf_BR = BinaryRelevance(#treat each label as a separate single-class classification problems
    classifier = model,
    require_dense = [False, True]
)

clf_CC = ClassifierChain(#treat each label as a park of a conditioned chain of single-class classificatn prob
    classifier = model,
    require_dense = [False, True]
)


clf_LP = LabelPowerset(
    classifier = model,
    require_dense = [False, True]
)

In [ ]:
clf_BR.fit(X_train,y_train)

In [ ]:
pickle.dump(clf_BR,open('/content/drive/My Drive/BT4221/Models/BR_SVM_v3.sav','wb'))

In [ ]:
clf_CC.fit(X_train,y_train)

In [ ]:
pickle.dump(clf_CC,open('/content/drive/My Drive/BT4221/Models/CC_SVM_v3.sav','wb'))

In [ ]:
clf_LP.fit(X_train,y_train)

In [ ]:
pickle.dump(clf_LP,open('/content/drive/My Drive/BT4221/Models/LP_SVM_v3.sav','wb'))